# SVM

本文参考了[知乎文章](https://zhuanlan.zhihu.com/p/77750026)以及李航的<<统计学习方法>>

本笔记没有记录SVM任何参数优化的算法。如果需要了解，可以参考[SMO](https://zhuanlan.zhihu.com/p/29212107)以及李航的<<统计学习方法>>,都有详细的公式推导过程。

## 线性可分

定义：$D_0$和$D_1$是 n 维欧氏空间中的两个点集。如果存在n维向量$W$和实数 b，使得所有属于$D_0$的点$x_i$都有$Wx_i + b > 0$，而对于所有属于$D_0$的点$x_j$则有$Wx_i + b > 0$，则我们称$D_0$和$D_1$线性可分。

## 最大间隔超平面

从二维扩展到多维空间中时，将$D_0$和$D_1$完全正确地划分开的$Wx + b = 0$就成了一个超平面。
在进行分类任务的时候，我们会去找最佳超平面，以最大间隔把两类样本分开的超平面，也称之为最大间隔超平面。

- 两类样本分别分割在该超平面的两侧；
- 两侧距离超平面最近的样本点到超平面的距离被最大化了。

同时，样本中距离超平面最近的店也被称为**支持向量**。通常在样本中，支持向量的数目并不是很多。最大间隔超平面是唯一的，证明见李航《统计学习方法》

## SVM的目标

对于给定数据集和超平面，定义样本点$(x_i, y_i)$与超平面的函数间隔为
$$
\hat{\varUpsilon_i} = y_i(Wx_i + b)\tag{1}
$$
那么定义数据集中所有样本点到超平面的最小函数间隔为
$$
\hat{\varUpsilon} = min\ \hat{\varUpsilon_i}\ i = 1,\ldots, N
\tag{2}
$$
而样本点到函数的几何间隔为
$$
\varUpsilon = \frac{\hat{\varUpsilon}}{||W||}\tag{3}
$$
**如果超平面的参数$W, b$** 成比例改变，其样本点的函数间隔也按比例改变，但是几何间隔不发生变化。

由于SVM就是为了找到最大间隔超平面，那么其目标函数可以写作如下形式。
$$
\begin{aligned}
\underset{w, b}{\max}\ \ &\frac{\hat{\varUpsilon}}{||w||}\\
s.t.\  \ \ &y_i(w\cdot x_i + b) \ge \hat\varUpsilon &\\
\end{aligned}
\tag{4}
$$

由于函数间隔的大小不影响几何间隔的大小，我们可以直接设定函数间隔为1，那么原优化问题可以改写为如下形式
$$
\begin{aligned}
\underset{w, b}{\min}\ \ &\frac{1}{2}||w||^2\\
s.t.\  \ \ &y_i(w\cdot x_i + b) -1 \ge 0 &\\
\end{aligned}
\tag{5}
$$
这是一个**凸二次规划**问题。求解该问题就可以获得最优的超平面的参数，也就解决了问题(该问题直接求解复杂度较高)。

## SVM的对偶问题

为了求解SVM的最优化问题，应用拉格朗日对偶性，通过求解对偶问题，得到原始问题的最优解。这样做的目的有两个

1. 对偶问题更容易求解
2. 自然引入核函数

构造拉格朗如乘子
$$
L(w, b, \alpha) = \frac{1}{2}||w||^2 - \displaystyle\sum_{i = 1}^N\alpha_iy_i(w\cdot x_i + b) + \sum_{i = 1}^N\alpha_i
\tag{6}
$$
其中$\alpha = (\alpha_1, \ldots, \alpha_N)^T$为拉格朗日乘子向量。

根据拉格朗日对偶性，SVM原始问题的对偶问题是极大极小问题
$$
\underset{\alpha}{\max}\underset{w, b}{\min}L(w, b, \alpha)\tag{7}
$$
先求解极小问题，将拉格朗日函数分别对$w, b$求偏导数并令其为0，可以得到如下两个表达式
$$
w = \displaystyle\sum_{i = 1}^N\alpha_iy_ix_i\tag{8}
$$

$$
\displaystyle\sum_{i = 1}^N \alpha_iy_i = 0\tag{9}
$$

将(8), (9)结合(6)(7)中，得到
$$
\underset{w, b}{\min}L(w, b, \alpha) = -\frac{1}{2}
\displaystyle\sum_{i = 1}^N\sum_{j = 1}^N\alpha_i\alpha_jy_iy_j(x_i\cdot x_j) + \sum_{i = 1}^N\alpha_i\tag{10}
$$

那么，求对$\underset{w, b}{\min}L(w, b, \alpha)$对$\alpha$的极大，即对偶问题
$$
\begin{aligned}
\underset{\alpha}{\max}&\ -\frac{1}{2}
\displaystyle\sum_{i = 1}^N\sum_{j = 1}^N\alpha_i\alpha_jy_iy_j(x_i\cdot x_j) + \sum_{i = 1}^N\alpha_i\\
s. t.\ &\ \displaystyle\sum_{i = 1}^N\alpha_iy_i = 0\\
& \alpha_i \ge 0, \ i = 1, 2, \ldots, N
\end{aligned}\tag{11}
$$

将目标函数变成求极小
$$
\begin{aligned}
\underset{\alpha}{\min}&\ \frac{1}{2}
\displaystyle\sum_{i = 1}^N\sum_{j = 1}^N\alpha_i\alpha_jy_iy_j(x_i\cdot x_j) - \sum_{i = 1}^N\alpha_i\\
s. t.\ &\ \displaystyle\sum_{i = 1}^N\alpha_iy_i = 0\\
& \alpha_i \ge 0, \ i = 1, 2, \ldots, N
\end{aligned}\tag{12}
$$

假设$\alpha^* = (\alpha^*_1, \alpha^*_2,\ldots,\alpha^*_N)^T$是对偶问题(12)的解，则存在下标$j$, 使得$\alpha^*_j > 0$(对应的$x_i$就是支持向量)，并可按照下式求解原问题
$$
\begin{aligned}
w^* &= \displaystyle\sum_{i = 1}^N\alpha_i^*y_ix_i\\
\end{aligned} \tag{13}
$$

$$
b^* = y_i - \displaystyle\sum_{i = 1}^N\alpha_i^*y_ix_i \tag{14}
$$

## 软间隔最大化
在回到上面的问题，上面求得的超平面只能够对线性可分的数据进行分类，一旦数据不线性可分的时候，即存在噪声的时候，应该如何处理？

软间隔最大化就是为了解决这个为题，当存在噪声的时候，函数间隔(1)不能每时每刻都为正，那么，就需要添加一个松弛量$\xi$。添加松弛量$\xi$之后，目标函数需要为每一个松弛变量支付代价，那么公式(5)变成如下形式
$$
\begin{aligned}
\underset{w, b}{\min}\ \ &\frac{1}{2}||w||^2 + C\displaystyle\sum_{i = 1}^N\xi_i\\
s.t.\  \ \ &y_i(w\cdot x_i + b) \ge 1 - \xi_i \\
& \xi_i \ge 0
\end{aligned}
\tag{15}
$$

问题(15)的对偶问题，可以表示成。
$$
\begin{aligned}
\underset{\alpha}{\min}&\ \frac{1}{2}
\displaystyle\sum_{i = 1}^N\sum_{j = 1}^N\alpha_i\alpha_jy_iy_j(x_i\cdot x_j) - \sum_{i = 1}^N\alpha_i\\
s. t.\ &\ \displaystyle\sum_{i = 1}^N\alpha_iy_i = 0\\
& 0 \le \alpha_i \le C, \ i = 1, 2, \ldots, N
\end{aligned}\tag{16}
$$

## 核技巧
核技巧是SVM在机器学习领域出类拔萃的主要原因。有一些数据样本，是通过软间隔最大化也无法解决的，能够正确对其分类需要一个闭合功能。这显然增加了求解的难度。核技巧的作用就是将这类样本映射到高维空间，在高维空间，数据就线性可分了。

核函数定义如下：

​	设$\chi$设输入空间，又设$\varPi$为特征空间，如果存在一个从$\chi$到$\varPi$的映射
$$
\phi(x): \chi \to \varPi\tag{17}
$$
使得所有$x, z \in \chi$，核函数满足条件
$$
K(x, z) = \phi(x)\cdot\phi(z)\tag{18}
$$
则称$K(x, z)$为核函数， $\phi(x)$为映射函数。

## 核技巧在SVM中的应用

在对偶问题中，目标函数(16)中的内积$x_i\cdot x_j$只可以用核函数$K(x_i, x_j)$来代替。对偶问题的目标函数就变成了
$$
W(\alpha) =\frac{1}{2}\displaystyle\sum_{i = 1}^N\sum_{j = 1}^N\alpha_i\alpha_jy_iy_jK(x_i, x_j) - \sum_{i = 1}^N\alpha_i\tag{17}
$$